In [26]:
import pandas as pd

# Create the DataFrame
data = [
    ["Sunny", "Warm", "Normal", "Strong", "Warm", "Same", "Yes"],
    ["Sunny", "Warm", "High", "Strong", "Warm", "Same", "Yes"],
    ["Rainy", "Cold", "High", "Strong", "Warm", "Change", "No"],
    ["Sunny", "Warm", "High", "Strong", "Cool", "Change", "Yes"]
]
columns = ["Sky", "AirTemp", "Humidity", "Wind", "Water", "Forecast", "EnjoySport"]
df = pd.DataFrame(data, columns=columns)

# Define the attribute values
attribute_values = {
    "Sky": ["Sunny", "Rainy","Cloudy"],
    "AirTemp": ["Warm", "Cold"],
    "Humidity": ["Normal", "High"],
    "Wind": ["Strong"],
    "Water": ["Warm", "Cool"],
    "Forecast": ["Same", "Change"]
}

# Define attribute mapping
attributes_map = {
    "Sky": 0,
    "AirTemp": 1,
    "Humidity": 2,
    "Wind": 3,
    "Water": 4,
    "Forecast": 5
}

# Initialize hypothesis functions
def g_0(n):
    return ("?",) * n

def s_0(n):
    return ('0',) * n

def more_general(h1, h2):
    return all(x == "?" or (x == y or y == "0") for x, y in zip(h1, h2))

def covers(example, hypothesis):
    return more_general(hypothesis, example)

def min_generalizations(h, x):
    h_new = list(h)
    for i in range(len(h)):
        if not covers(x[i:i+1], h[i:i+1]):
            h_new[i] = '?' if h[i] != '0' else x[i]
    return [tuple(h_new)]

def min_specializations(h, domains, x):
    results = []
    for i in range(len(h)):
        if h[i] == "?":
            for val in domains[i]:
                if x[i] != val:
                    h_new = h[:i] + (val,) + h[i+1:]
                    results.append(h_new)
        elif h[i] != "0":
            h_new = h[:i] + ('0',) + h[i+1:]
            results.append(h_new)
    return results

def candidate_elimination(examples):
    domains = list(attribute_values.values())
   
    G = {g_0(len(domains))}
    S = {s_0(len(domains))}
   
    i = 0
    print(f"\n G[{i}]:", G)
    print(f"\n S[{i}]:", S)
   
    for xcx in examples:
        i += 1
        x, cx = xcx[:-1], xcx[-1]  # Splitting data into attributes and decisions
       
        if cx == 'Yes':  # x is positive example
            G = {g for g in G if covers(x, g)}
            S = generalize_S(x, G, S)
        else:  # x is negative example
            S = {s for s in S if not covers(x, s)}
            G = specialize_G(x, domains, G, S)
       
        print(f"\n G[{i}]:", G)
        print(f"\n S[{i}]:", S)
   
    return

def generalize_S(x, G, S):
    S_prev = list(S)
    for s in S_prev:
        if s not in S:
            continue
        if not covers(x, s):
            S.remove(s)
            Splus = min_generalizations(s, x)
            S.update([h for h in Splus if any([more_general(g, h) for g in G])])
            S.difference_update([h for h in S if any([more_general(h, h1) for h1 in S if h != h1])])
    return S

def specialize_G(x, domains, G, S):
    G_prev = list(G)
    for g in G_prev:
        if g not in G:
            continue
        if covers(x, g):
            G.remove(g)
            Gminus = min_specializations(g, domains, x)
            G.update([h for h in Gminus if any([more_general(h, s) for s in S])])
            G.difference_update([h for h in G if any([more_general(g1, h) for g1 in G if h != g1])])
    return G

# Convert DataFrame to list of tuples
examples = df.apply(tuple, axis=1).tolist()
candidate_elimination(examples)







 G[0]: {('?', '?', '?', '?', '?', '?')}

 S[0]: {('0', '0', '0', '0', '0', '0')}

 G[1]: {('?', '?', '?', '?', '?', '?')}

 S[1]: {('Sunny', 'Warm', 'Normal', 'Strong', 'Warm', 'Same')}

 G[2]: {('?', '?', '?', '?', '?', '?')}

 S[2]: {('Sunny', 'Warm', '?', 'Strong', 'Warm', 'Same')}

 G[3]: {('?', 'Warm', '?', '?', '?', '?'), ('Sunny', '?', '?', '?', '?', '?'), ('?', '?', '?', '?', '?', 'Same')}

 S[3]: {('Sunny', 'Warm', '?', 'Strong', 'Warm', 'Same')}

 G[4]: {('?', 'Warm', '?', '?', '?', '?'), ('Sunny', '?', '?', '?', '?', '?')}

 S[4]: {('Sunny', 'Warm', '?', 'Strong', '?', '?')}
